# SageMaker 환경 정리하기

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
        <b>이 노트북은 환경에서 실행한 모든 노트북에서 생성된 모든 리소스를 제거합니다.</b>
    </p>
</div>

❗ 다음 코드 셀은:
- Studio 환경에서 프로비저닝한 프로젝트를 영구적으로 삭제합니다
- 피처 그룹을 영구적으로 삭제합니다
- 프로젝트가 프로비저닝한 S3 버킷을 영구적으로 삭제합니다
- 프로젝트 관련 접두사 아래의 S3 버킷 내 객체를 영구적으로 삭제합니다
- 추론 엔드포인트를 영구적으로 삭제합니다

<div class="alert alert-info"> ❗ 프로젝트, 피처 그룹 및 엔드포인트 목록을 보여주는 코드 셀은 <b>이 워크숍 외부</b>에서 생성된 것들을 포함하여 AWS 계정의 모든 SageMaker 리소스를 표시합니다. 자신의 리소스를 실수로 삭제하지 않도록 삭제할 리소스의 이름을 다시 확인하세요.
</div>

<div class="alert alert-info"> 💡 <strong> 이 노트북은 AWS 계정의 리소스를 영구적으로 삭제합니다. 삭제할 리소스의 이름을 다시 확인해주세요! </strong>
</div>

<div class="alert alert-info">강사 주도 워크숍에서 AWS가 제공한 AWS 계정을 사용하는 경우 이 노트북을 실행할 필요가 없습니다.</div>

<div class="alert alert-info"> 이 노트북에서 JupyterLab의 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
import sagemaker
import boto3
import time
import json
import os

In [ ]:
%store -r 
%store

In [ ]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

## 프로젝트 삭제하기

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

In [ ]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

In [ ]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"The following projects will be deleted:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")

In [ ]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

## CloudFormation 스택 삭제하기
이 섹션에서는 프로젝트가 생성한 AWS CloudFormation 스택을 삭제합니다

In [ ]:
cfn = boto3.client("cloudformation")

for p in projects_to_delete:
    for s in [
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-staging",
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-prod"
            ]:
        try:
            print(f"Delete CloudFormation stack: {s}")
            r = cfn.delete_stack(StackName=s)
            print(r)
            time.sleep(180)
        except Exception as e:
            print(f"Exception in deleting {s}:{e}")
            pass

## 피처 그룹 삭제하기

In [ ]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [ ]:
feature_groups

In [ ]:
# Select feature groups to be deleted
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Are you sure you want to delete this feature group: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"The following feature groups will be deleted:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")

In [ ]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'Feature group: {feature_group_name} does NOT have an offline store!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"all feature store objects under {offline_store_s3_uri} will be deleted!")
    print("Are you sure you want to these objects ? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive

<div class="alert alert-info"> 💡 <strong> 다음 코드 셀은 선택한 피처 그룹을 삭제합니다!</strong>
</div>

In [ ]:
for fg in feature_groups_to_delete:
    print(f"Deleting the feature group: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)

## 프로젝트가 프로비저닝한 S3 버킷 삭제하기

<div class="alert alert-info"> 💡 <strong> 다음 코드 셀은 프로젝트가 생성한 모든 S3 버킷을 삭제합니다!</strong> 강사 주도 워크숍에 참여하고 제공된 AWS 계정을 사용하는 경우, S3 버킷을 삭제할 수 없을 수 있습니다. 여기서 중단하셔도 됩니다.
</div>

In [ ]:
print(f"*****************************************************")
print(f"The following S3 buckets will be removed permanently!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-project-{p['ProjectId']}")

In [ ]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-project-{p['ProjectId']} --force 

## 추론 엔드포인트 제거하기

In [ ]:
endpoints = sm.list_endpoints()["Endpoints"]

In [ ]:
endpoints_to_delete = []

for ep in endpoints:
    print(f"Are you sure you want to delete this endpoint: {ep['EndpointName']}? (y/n)")
    choice = input()
    if choice == 'y':
        endpoints_to_delete.append(ep['EndpointName'])
        
print(f"*********** THESE ENDPOINTS WILL BE DELETED ***********")
print('\n'.join(endpoints_to_delete))
print(f"*******************************************************")

In [ ]:
for ep in endpoints_to_delete:
    try:
        delete_mon_schedules(ep, sm)
        time.sleep(10)
        print(f"Deleting the endpoint: {ep}:{sm.delete_endpoint(EndpointName=ep)}")
    except Exception as e:
        print(f"Exception in deleting {ep}:{e}")
        pass

## SageMaker S3 데이터 버킷에서 프로젝트 관련 객체 제거하기

<div class="alert alert-info"> 💡 <strong> 다음 코드 셀은 지정된 S3 접두사 아래의 모든 객체를 삭제합니다!</strong>
</div>

In [ ]:
prefixes_to_delete = [
    bucket_prefix,
    #s3_fs_query_output_prefix
]

In [ ]:
print(f"************************************************************************")
print(f"All objects under the following S3 prefixes will be removed permanently!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(f"{bucket_name}/{p}")

<div class="alert alert-info"> ❗ <strong> S3 접두사를 다시 확인하세요! 이 접두사 아래의 모든 데이터가 영구적으로 삭제됩니다!</strong>
</div>

In [ ]:
# Un-comment the rm command
for p in prefixes_to_delete:
    # !aws s3 rm s3://{bucket_name}/{p} --recursive
    pass

## MLflow 서버 삭제하기
생성된 MLflow 추적 서버는 삭제하거나 중지할 때까지 비용이 발생합니다. 추적 서버에 대한 요금은 서버가 실행된 기간, 선택한 크기 및 추적 서버에 기록된 데이터의 양을 기준으로 합니다.

추가 비용을 방지하려면 수동으로 생성한 모든 MLFlow 추적 서버를 삭제해야 합니다.

<div class="alert alert-info">UX나 API를 통해 수동으로 추적 서버를 생성하지 않았다면 MLflow 추적 서버를 삭제하고 해당 셀을 실행할 필요가 없습니다.</div>

In [ ]:
# Find all active MLflow servers in the domain
tracking_servers = sm.list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

ts_to_delete = []

for ts in tracking_servers:
    print(f"Are you sure you want to delete this MLflow tracking server: {ts['TrackingServerName']}? (y/n)")
    choice = input()
    if choice == 'y':
        ts_to_delete.append(ts['TrackingServerName'])

print(f"*********** THESE MLFLOW TRACKING SERVERS WILL BE DELETED ***********")
print('\n'.join(ts_to_delete))
print(f"**********************************************************************")

In [ ]:
for ts in ts_to_delete:
    try:
        print(f"Deleting the MLflow tracking server: {ts}:{sm.delete_mlflow_tracking_server(TrackingServerName=ts)}")
    except Exception as e:
        print(f"Exception in deleting {ts}:{e}")
        pass

# 커널 종료하기

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>